In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from random import sample
from random import choice
from IPython.display import IFrame

In [2]:
df = pd.read_csv('students.csv')

In [52]:
df_aleks_test = pd.read_csv('students_test.csv')

In [3]:
df.head()

,student_id,student__first_name,country,age,has_rabbit
0,1,David,Brazil,31,True
1,2,Ahmet,Canada,20,True
2,3,Isaac,USA,20,True
3,4,Charlie,Indonesia,24,True
4,5,Isaac,USA,35,True


In [4]:
df_countries = df.country.value_counts().to_dict()
df_countries

{'Mexico': 153,
 'Canada': 141,
 'Brazil': 129,
 'Portugal': 86,
 'Indonesia': 79,
 'Turkey': 76,
 'Poland': 73,
 'Argentina': 71,
 'China': 67,
 'USA': 63,
 'Cambodja': 62}

In [5]:
treshold = len(df)/2

#finding the country with the highest amount of students and removing it from the dictionary
biggest_country = [max(df_countries, key=df_countries.get),df_countries.pop(max(df_countries, key=df_countries.get))]

# sets the counter to the value of students of the biggest country
counter = biggest_country[1]

df1_list = []
df2_list = []
df1_list.append(biggest_country) #adding the key : value pair to the df1_list before iterating
for country,students in df_countries.items():
    item = [country,students]
    if counter < treshold:
        df1_list.append(item)
        # adding countries up to the treshold
    else:
        df2_list.append(item)
        # adding the rest to the other df
    counter += students

In [6]:
df1_list

[['Mexico', 153], ['Canada', 141], ['Brazil', 129], ['Portugal', 86]]

In [7]:
df1_columns = pd.DataFrame(data=df1_list,columns=['country', 'students'])
display(df1_columns)
print(f'Sum of students is: {df1_columns.students.sum()}')

,country,students
0,Mexico,153
1,Canada,141
2,Brazil,129
3,Portugal,86


Sum of students is: 509


In [8]:
df2_columns = pd.DataFrame(data=df2_list,columns=['country', 'students'])
display(df2_columns)
print(f'Sum of students is: {df2_columns.students.sum()}')

,country,students
0,Indonesia,79
1,Turkey,76
2,Poland,73
3,Argentina,71
4,China,67
5,USA,63
6,Cambodja,62


Sum of students is: 491


In [9]:
df1_columns.students.sum()

509

In [10]:
c1 = df1_columns.country.tolist()
df1 = df[df.country.isin(c1)]
df1

,student_id,student__first_name,country,age,has_rabbit
0,1,David,Brazil,31,True
1,2,Ahmet,Canada,20,True
9,10,Frank,Portugal,35,True
10,11,Huan-Chan,Mexico,33,False
11,12,Miguel,Mexico,27,True
...,...,...,...,...,...
994,995,Ahmet,Brazil,38,False
995,996,David,Portugal,34,True
996,997,Grace,Canada,23,False
997,998,Emma,Canada,26,False


In [11]:
df2_columns = pd.DataFrame(data=df2_list,columns=['country', 'students'])
df2_columns.head()

,country,students
0,Indonesia,79
1,Turkey,76
2,Poland,73
3,Argentina,71
4,China,67


In [12]:
df2_columns.students.sum()

491

In [13]:
c2 = df2_columns.country.tolist()
df2 = df[df.country.isin(c2)]
df2

,student_id,student__first_name,country,age,has_rabbit
2,3,Isaac,USA,20,True
3,4,Charlie,Indonesia,24,True
4,5,Isaac,USA,35,True
5,6,Miguel,Argentina,37,True
6,7,Julia,Poland,28,False
...,...,...,...,...,...
984,985,Pedro,USA,38,False
985,986,Bob,Cambodja,33,False
990,991,Ahmet,Turkey,32,True
992,993,David,Turkey,29,False


# Test with random DF

##### set the number of countries, randomly pick that amount from the options list and from that list chose a country that will have the most samples 

In [14]:
country_options = ['Brazil','Canada','Mexico','USA','Turkey','Germany','France','Netherlands','Japan','Russia','Nigeria']
countries_choice = np.random.choice(country_options,8,replace=False).tolist()
countries_choice
print(len(countries_choice))
skewed_country = sample(countries_choice,k=1)[0]
print(skewed_country)

8
Nigeria


In [34]:
def create_random_df(number_of_rows,percent_of_skewed_country_entries,number_of_countries) -> pd.DataFrame:
    from random import sample
    from random import choice
    # creating the empty lists and the append options for each column
    students = []

    student_first_name = []
    name_options = ['Huey','Dewey','Louie']

    # set the number of countries, randomly pick that amount from the options list 
     
    country = []
    country_options = ['Brazil','Canada','Mexico','USA','Turkey','Germany',
                       'France','Netherlands','Japan','Russia','Nigeria']
    countries_choice = np.random.choice(country_options,number_of_countries,replace=False).tolist()
    
    age = []
    age_options = [*range(18,35)]

    has_rabbit = []
    rabbit = [True,False]
    
    # and from that list chose a country that will have the most samples
    skewed_country = sample(countries_choice,k=1)[0]
    print(f'The skewed country is: {skewed_country}')
    countries_choice.remove(skewed_country)
    
    
    for i in range(number_of_rows):
        # iterating the determined times to append items to the lists 
        students.append(i)
        student_first_name.append(sample(name_options,k=1)[0])
        age.append(sample(age_options,k=1)[0])
        has_rabbit.append(sample(rabbit,k=1)[0])
        
        # filling lists with non - skewed countries
        if i < ((number_of_rows/100)*(100-percent_of_skewed_country_entries)):
            country.append(sample(countries_choice,k=1)[0])
        # fill the rest with skewed_country
        else: country.append(skewed_country)
    
    # create the df
    test_df = pd.DataFrame({
            'student_id' : students,
    'student_first_name' : student_first_name,
               'country' : country,
                   'age' : age,
            'has_rabbit' : has_rabbit,
})
    return test_df

In [35]:
def split_df(df: pd.DataFrame) -> pd.DataFrame:
    
    # creating dictionary to create a boolean mask
    df_countries = df.country.value_counts().to_dict()
    
    treshold = len(df)/2

    # finding the country with the highest amount of students and removing it from the dictionary
    biggest_country = [max(df_countries, key=df_countries.get),df_countries.pop(max(df_countries, key=df_countries.get))]
    
    # sets the counter to the value of students of the biggest country
    counter = biggest_country[1]

    df1_list = []
    df2_list = []
    df1_list.append(biggest_country) #adding the key : value pair to the df1_list before iterating
    for country,students in df_countries.items():
        item = [country,students]
        
        if counter < treshold:
            # adding countries up to the treshold to df1_list
            df1_list.append(item)
            
        else:
            # adding the rest to the other df2_list
            df2_list.append(item)
            
        counter += students
    
    # create boolean mask for df1
    df1_columns = pd.DataFrame(data=df1_list,columns=['country', 'students'])
    c1 = df1_columns.country.tolist()
    
    # divide df1 using boolean mask
    df1 = df[df.country.isin(c1)]
    df1 = df1.reset_index(drop=True)
    
    # create boolean mask for df2
    df2_columns = pd.DataFrame(data=df2_list,columns=['country', 'students'])
    c2 = df2_columns.country.tolist()
    
    # divide df1 using boolean mask
    df2 = df[df.country.isin(c2)]
    df2 = df2.reset_index(drop=True)
    return [df1,df2]

In [45]:
test1_df,test2_df = split_df(create_random_df(5000,50,8))

The skewed country is: Germany


In [70]:
shola = create_random_df(10000,90,10)

The skewed country is: Nigeria


In [71]:
shola

,student_id,student_first_name,country,age,has_rabbit
0,0,Louie,Mexico,28,False
1,1,Louie,Brazil,24,True
2,2,Huey,Germany,34,True
3,3,Huey,Brazil,21,False
4,4,Louie,Brazil,32,True
...,...,...,...,...,...
9995,9995,Louie,Nigeria,29,True
9996,9996,Louie,Nigeria,19,False
9997,9997,Louie,Nigeria,31,False
9998,9998,Huey,Nigeria,33,True


In [72]:
shola.to_csv('shola.csv', index=False)

In [46]:
test1_df

,student_id,student_first_name,country,age,has_rabbit
0,2500,Huey,Germany,30,False
1,2501,Huey,Germany,26,False
2,2502,Dewey,Germany,34,True
3,2503,Louie,Germany,19,False
4,2504,Louie,Germany,19,False
...,...,...,...,...,...
2495,4995,Louie,Germany,21,False
2496,4996,Huey,Germany,22,False
2497,4997,Dewey,Germany,23,True
2498,4998,Huey,Germany,25,True


In [47]:
test1_df.country.value_counts()

Germany    2500
Name: country, dtype: int64

In [48]:
len(test1_df.country.value_counts())

1

In [49]:
test2_df

,student_id,student_first_name,country,age,has_rabbit
0,0,Louie,Canada,26,False
1,1,Huey,Turkey,29,True
2,2,Dewey,Russia,28,True
3,3,Huey,Nigeria,29,False
4,4,Huey,Turkey,18,True
...,...,...,...,...,...
2495,2495,Dewey,Mexico,28,True
2496,2496,Louie,USA,22,False
2497,2497,Louie,Turkey,22,False
2498,2498,Dewey,Mexico,31,False


In [50]:
test2_df.country.value_counts()

Nigeria    387
Russia     371
France     358
Turkey     357
USA        348
Canada     343
Mexico     336
Name: country, dtype: int64

In [51]:
len(test2_df.country.value_counts())

7

## Testing area

In [43]:
test_single_df = create_random_df(1000,30,8)

The skewed country is: France


In [44]:
test_single_df.country.value_counts()

France         300
Canada         115
Germany        107
Japan          103
Netherlands    102
Mexico          99
Russia          88
USA             86
Name: country, dtype: int64

In [53]:
test1_aleks,test2_aleks = split_df(df_aleks_test)

In [54]:
test1_aleks

,student_id,student__first_name,country,age,has_rabbit
0,2,Ahmet,Indonesia,21,True
1,3,Anton,Indonesia,18,False
2,4,Frank,Indonesia,35,False
3,6,David,Canada,19,True
4,8,Isaac,Brazil,33,False
...,...,...,...,...,...
2586,4992,Huan-Chan,Indonesia,31,True
2587,4993,Andreia,Turkey,38,False
2588,4994,Hannah,Brazil,25,False
2589,4998,Hannah,Canada,22,True


In [55]:
test2_aleks

,student_id,student__first_name,country,age,has_rabbit
0,1,Julia,Peru,20,True
1,5,Huan-Chan,Panama,35,False
2,7,Bob,Honduras,22,False
3,10,Guillaume,China,33,True
4,11,Miguel,Peru,21,True
...,...,...,...,...,...
2404,4991,Ahmet,Slovakia,29,True
2405,4995,Emma,Cambodja,33,True
2406,4996,Grace,Poland,23,False
2407,4997,Alice,China,38,False


In [56]:
test1_aleks.country.value_counts()

Canada        466
Brazil        455
Mexico        455
Indonesia     257
Japan         243
Argentina     240
Kazakhstan    239
Turkey        236
Name: country, dtype: int64

In [57]:
test2_aleks.country.value_counts()

Kyrgystan    235
Peru         230
Panama       227
USA          226
Honduras     225
China        218
Hungary      216
Poland       216
Slovakia     206
Cambodja     205
Portugal     205
Name: country, dtype: int64

In [73]:
shola1,shola2 = split_df(shola)

In [74]:
shola1

,student_id,student_first_name,country,age,has_rabbit
0,1000,Dewey,Nigeria,31,False
1,1001,Louie,Nigeria,24,False
2,1002,Huey,Nigeria,33,False
3,1003,Huey,Nigeria,27,True
4,1004,Huey,Nigeria,30,True
...,...,...,...,...,...
8995,9995,Louie,Nigeria,29,True
8996,9996,Louie,Nigeria,19,False
8997,9997,Louie,Nigeria,31,False
8998,9998,Huey,Nigeria,33,True


In [75]:
shola2

,student_id,student_first_name,country,age,has_rabbit
0,0,Louie,Mexico,28,False
1,1,Louie,Brazil,24,True
2,2,Huey,Germany,34,True
3,3,Huey,Brazil,21,False
4,4,Louie,Brazil,32,True
...,...,...,...,...,...
995,995,Huey,Mexico,18,True
996,996,Dewey,Netherlands,29,False
997,997,Louie,Mexico,25,False
998,998,Louie,Mexico,21,False
